In [35]:
import numpy as np
import tensorflow as tf

print(tf.__version__)

2.11.0


In [36]:

from pathlib import Path

local_dir = Path().cwd()
train_dir = local_dir / 'data/train/'
test_dir = local_dir / 'data/test/'
val_dir = local_dir / 'data/val/'

#Reads the data from disk
train_ds = tf.keras.utils.image_dataset_from_directory(train_dir, image_size=(224, 224), seed=123)
val_ds = tf.keras.utils.image_dataset_from_directory(val_dir, image_size=(224, 224), seed=123)
test_ds = tf.keras.utils.image_dataset_from_directory(test_dir, image_size=(224, 224), seed=123)

print(train_ds.class_names)
print(val_ds.class_names)
print(test_ds.class_names)



Found 3045 files belonging to 2 classes.
Found 16 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
['NORMAL', 'PNEUMONIA']
['NORMAL', 'PNEUMONIA']
['NORMAL', 'PNEUMONIA']


In [37]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical", seed=123),
    tf.keras.layers.RandomRotation(0.2)
])

In [38]:
# prints the shape of the training dataset
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 224, 224, 3)
(32,)


In [39]:
# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [40]:
model = tf.keras.models.load_model('tfmodel')

In [41]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
])
aug_train_ds = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y),
  num_parallel_calls=AUTOTUNE
)

In [42]:
# Training the network
model.fit(
  aug_train_ds,
  validation_data=val_ds,
  batch_size=64,
  epochs=40
)

Epoch 1/40
96/96 [==============================] - 398s 4s/step - loss: 0.4378 - accuracy: 0.8634 - val_loss: 0.1387 - val_accuracy: 0.8750
Epoch 2/40
96/96 [==============================] - 396s 4s/step - loss: 0.1897 - accuracy: 0.9327 - val_loss: 0.1173 - val_accuracy: 0.9375
Epoch 3/40
96/96 [==============================] - 390s 4s/step - loss: 0.1622 - accuracy: 0.9402 - val_loss: 0.0595 - val_accuracy: 1.0000
Epoch 4/40
96/96 [==============================] - 398s 4s/step - loss: 0.1610 - accuracy: 0.9389 - val_loss: 0.1443 - val_accuracy: 0.9375
Epoch 5/40
96/96 [==============================] - 399s 4s/step - loss: 0.1538 - accuracy: 0.9471 - val_loss: 0.0823 - val_accuracy: 0.9375
Epoch 6/40
96/96 [==============================] - 394s 4s/step - loss: 0.1254 - accuracy: 0.9521 - val_loss: 0.0502 - val_accuracy: 1.0000
Epoch 7/40
96/96 [==============================] - 389s 4s/step - loss: 0.1190 - accuracy: 0.9530 - val_loss: 0.1558 - val_accuracy: 0.9375
Epoch 8/40
96

In [43]:
score = model.evaluate(test_ds)

20/20 [==============================] - 20s 941ms/step - loss: 0.8558 - accuracy: 0.8365


In [44]:

print(score)

[0.8558077812194824, 0.8365384340286255]


In [45]:
model.save('tfmodel_2')

INFO:tensorflow:Assets written to: tfmodel_2\assets


INFO:tensorflow:Assets written to: tfmodel_2\assets
